In [ ]:
# !pip install google-cloud-storage requests

In [20]:
!set GOOGLE_APPLICATION_CREDENTIALS="F:\\setup files\\downloads\\totemic-program-442307-i9-cdc0176778ce.json"

In [19]:
ls

 Volume in drive F is New Volume
 Volume Serial Number is EC26-6E9F

 Directory of f:\CodingEnvironment\BigData\flow

12/08/2024  04:55 PM    <DIR>          .
12/08/2024  04:55 PM    <DIR>          ..
12/08/2024  04:46 PM     7,163,404,119 airline.tar.gz
12/08/2024  04:57 PM             1,668 to_gcs.py
12/08/2024  03:57 PM                 0 web_to_gcs.ipynb
               3 File(s)  7,163,405,787 bytes
               2 Dir(s)  44,627,509,248 bytes free


In [ ]:
import requests
import tarfile
from google.cloud import storage

# Define variables
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/airline.tar.gz'
download_path = 'airline.tar.gz'  # Local path to save the tar.gz file
bucket_name = 'bk9999airline'  # Replace with your GCS bucket name
blob_name = 'airline.csv'  # Name of the file in the GCS bucket

# Step 1: Download the file
def download_file(url, destination):
    print(f"Starting download from {url}...")
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Check if the request was successful
        total_size = int(response.headers.get('content-length', 0))  # Get file size
        chunk_size = 1024 * 1024  # 1 MB per chunk
        with open(destination, 'wb') as file:
            downloaded = 0
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:  # Filter out keep-alive chunks
                    file.write(chunk)
                    downloaded += len(chunk)
                    print(f"Downloaded {downloaded / total_size * 100:.2f}%")
    print(f"Download complete. File saved to {destination}.")

# Step 2: Extract and upload the file in chunks
def extract_and_stream_upload(file_path, bucket_name, blob_name):
    print(f"Extracting and uploading from {file_path} to GCS bucket: {bucket_name}/{blob_name}...")
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Open the tar.gz file
    with tarfile.open(file_path, 'r:gz') as tar:
        members = tar.getmembers()
        if len(members) == 1:
            member = members[0]
            print(f"Processing {member.name}...")
            
            # Read the extracted file in chunks and upload
            with tar.extractfile(member) as file_obj:
                chunk_size = 1024 * 1024 * 1024  
                while True:
                    chunk = file_obj.read(chunk_size)
                    if not chunk:
                        break
                    blob.upload_from_string(chunk, content_type='application/octet-stream', if_generation_match=None)
                    print(f"Uploaded {len(chunk)} bytes to GCS.")
        else:
            raise ValueError("The archive contains more than one file.")
    print(f"File successfully uploaded to gs://{bucket_name}/{blob_name}")

# Download the file
download_file(url, download_path)


Starting download from https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/airline.tar.gz...
Downloaded 0.01%
Downloaded 0.03%
Downloaded 0.04%
Downloaded 0.06%
Downloaded 0.07%
Downloaded 0.09%
Downloaded 0.10%
Downloaded 0.12%
Downloaded 0.13%
Downloaded 0.15%
Downloaded 0.16%
Downloaded 0.18%
Downloaded 0.19%


KeyboardInterrupt: 

In [17]:
extract_and_stream_upload(download_path, bucket_name, blob_name)

Extracting and uploading from airline.tar.gz to GCS bucket: bk9999_airline/airline.csv...


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [24]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="F:\\setup files\\downloads\\totemic-program-442307-i9-cdc0176778ce.json"

bucket_name = 'bk9999_airline'  # Replace with your GCS bucket name
blob_name = 'test.csv'  # Name of the file in the GCS bucket
destination_file_name = 'airline.tar.gz'  # Local path to save the file

def upload_blob(bucket_name, blob_name, destination_file_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_filename(destination_file_name)

    print(f"File {destination_file_name} uploaded to {blob_name}.")

In [25]:
upload_blob(bucket_name, blob_name, destination_file_name)

File airline.tar.gz uploaded to test.csv.


In [1]:
!python -m pip install prefect


In [2]:
conda deactivate



Note: you may need to restart the kernel to use updated packages.


In [4]:
!prefect version

'prefect' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
pip install prefect
